In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from warnings import filterwarnings

In [2]:
# Create order_period column based on order time
def get_order_period(order_time):
    if order_time >= pd.to_datetime('06:00:00').time() and order_time < pd.to_datetime('12:00:00').time():
        return 'Morning'
    elif order_time >= pd.to_datetime('12:00:00').time() and order_time < pd.to_datetime('18:00:00').time():
        return 'Afternoon'
    elif order_time >= pd.to_datetime('18:00:00').time() and order_time <= pd.to_datetime('23:59:59').time():
        return 'Night'
    else:
        return 'Night'
    
def get_week_of_month(date):
    first_day = date.replace(day=1)
    day_of_month = date.day
    adjusted_dom = day_of_month + first_day.weekday()
    return (adjusted_dom - 1) // 7 + 1

In [3]:
data = pd.read_csv('delhi_data.csv')
data.head()

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks
0,1,C8270,R2924,2024-02-01 01:11:52,2024-02-01 02:39:52,1914,0,Credit Card,5% on App,150,47,0
1,2,C1860,R2054,2024-02-02 22:11:04,2024-02-02 22:46:04,986,40,Digital Wallet,10%,198,23,0
2,3,C6390,R2870,2024-01-31 05:54:35,2024-01-31 06:52:35,937,30,Cash on Delivery,15% New User,195,45,0
3,4,C6191,R2642,2024-01-16 22:52:49,2024-01-16 23:38:49,1463,50,Cash on Delivery,NaN,146,27,0
4,5,C6734,R2799,2024-01-29 01:19:30,2024-01-29 02:48:30,1992,30,Cash on Delivery,50 off Promo,130,50,0


Feature Engineering

In [4]:
df = data.copy()

# Convert 'Order Date and Time' and 'Delivery Date and Time' to datetime format
df['Order Date and Time'] = pd.to_datetime(df['Order Date and Time'])
df['Delivery Date and Time'] = pd.to_datetime(df['Delivery Date and Time'])

# Extracting separate columns for order and delivery date and time
df['Order Date'] = df['Order Date and Time'].dt.date
df['Order Time'] = df['Order Date and Time'].dt.time
df['Delivery Date'] = df['Delivery Date and Time'].dt.date
df['Delivery Time'] = df['Delivery Date and Time'].dt.time

# Extract numerical value of discount from 'Discounts and Offers' column
df['Discount'] = df['Discounts and Offers'].str.extract(r"(\d+\.?\d*)").astype('Int8').fillna(0)

df['Order Period'] = df['Order Time'].apply(get_order_period)
df['Order Week'] = df['Order Date'].apply(get_week_of_month)

# Convert 'Order Week of Month' to an ordered categorical type
df['Order Week'] = df['Order Date and Time'].apply(lambda x: get_week_of_month(x))
week_categories = [1, 2, 3, 4, 5]
df['Order Week'] = pd.Categorical(df['Order Week'], categories=week_categories, ordered=True)

# Replace values in 'Payment Method' column
df['Payment Method'] = df['Payment Method'].replace({
    'Cash on Delivery': 'COD',
    'Credit Card': 'CC',
    'Digital Wallet': 'UPI'
})

df.sample(6)

,Order ID,Customer ID,Restaurant ID,Order Date and Time,Delivery Date and Time,Order Value,Delivery Fee,Payment Method,Discounts and Offers,Commission Fee,Payment Processing Fee,Refunds/Chargebacks,Order Date,Order Time,Delivery Date,Delivery Time,Discount,Order Period,Order Week
751,752,C2296,R2893,2024-01-04 13:01:34,2024-01-04 13:42:34,602,0,UPI,NaN,110,36,150,2024-01-04,13:01:34,2024-01-04,13:42:34,0,Afternoon,1
109,110,C1206,R2881,2024-01-14 14:05:30,2024-01-14 15:02:30,1788,50,CC,10%,187,32,0,2024-01-14,14:05:30,2024-01-14,15:02:30,10,Afternoon,2
400,401,C3489,R2974,2024-01-04 13:43:43,2024-01-04 14:26:43,287,50,COD,50 off Promo,122,26,0,2024-01-04,13:43:43,2024-01-04,14:26:43,50,Afternoon,1
985,986,C3717,R2317,2024-01-12 04:46:01,2024-01-12 06:33:01,910,50,CC,15% New User,122,11,150,2024-01-12,04:46:01,2024-01-12,06:33:01,15,Night,2
818,819,C2619,R2115,2024-01-08 23:32:36,2024-01-09 01:18:36,1331,30,CC,50 off Promo,196,38,50,2024-01-08,23:32:36,2024-01-09,01:18:36,50,Night,2
153,154,C4763,R2026,2024-01-12 08:54:29,2024-01-12 10:19:29,1305,20,CC,10%,88,16,0,2024-01-12,08:54:29,2024-01-12,10:19:29,10,Morning,2


In [5]:

## Top 5 valued Customers
cust_spend = df.groupby(['Customer ID']).agg({'Order Value':'sum'}).reset_index().sort_values(by='Order Value',ascending=False)
cust_spend.head()

,Customer ID,Order Value
730,C7949,3870
282,C3832,3768
770,C8266,3703
457,C5555,3664
406,C5146,3361


Customer wise analysis

In [8]:
import datetime as dt

In [7]:
cust_id = 'C7938'

In [77]:
customer_metrics = pd.DataFrame(columns=['Metric','Value'])
customer_metrics = customer_metrics.iloc[0:0]

In [78]:
# Assuming 'df' is the dfset containing customer information
current_date = dt.datetime.now()

# 1. Recency: Number of days since the customer's last order
recency = df.groupby('Customer ID')['Order Date and Time'].max().reset_index()
recency['Recency'] = (current_date - recency['Order Date and Time']).dt.days

df['Order Date and Time'] = pd.to_datetime(df['Order Date and Time'])
current_date = dt.datetime.now()

# 1. Recency: Number of days since the customer's last order
recency = df.groupby('Customer ID')['Order Date and Time'].max().reset_index()
recency['Recency'] = (current_date - recency['Order Date and Time']).dt.days

# Use .loc to filter the recency dfFrame for the given customer ID
customer_recency = recency[recency['Customer ID'] == cust_id][['Customer ID','Recency']].set_index(['Customer ID'])

customer_metrics = customer_metrics._append({'Customer ID': cust_id, 'Metric': 'Recency', 'Value': customer_recency['Recency'].values[0]}, ignore_index=True)
customer_metrics.head()


,Metric,Value,Customer ID
0,Recency,263,C7938
